In [ ]:
import sys
#sys.path.append('/home/marc_f/anaconda3/envs/default/bin/python/')

In [ ]:
sys.executable

In [1]:
import glob
import torch
import os,sys
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sb
import pandas as pd
import dill
import flow_ssl
from sklearn.model_selection import train_test_split
from flow_ssl.data import GAS, HEPMASS, MINIBOONE
from flow_ssl.data.nlp_datasets import AG_News,YAHOO
from torch.utils.data import DataLoader
from oil.datasetup.dataloaders import getLabLoader,classBalancedSampleIndices
from oil.utils.utils import FixedNumpySeed

In [304]:
H = HEPMASS(train=True)
M = MINIBOONE(train=True)
A = AG_News(train=True)
At = AG_News(train=False)
Ya = YAHOO(train=True)

In [3]:
def oh(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

In [4]:
# train_data = np.load('ag_news_train.npz')
# x_train = train_data['encodings']
# y_train = train_data['labels']
# test_data = np.load('ag_news_test.npz')
# x_test = test_data['encodings']
# y_test = test_data['labels']

In [198]:
test_size = 3000#3000

unlab_size=8000#5000

In [292]:
X_train = H.trn
X_test = H.tst
y_train = H.y_trn.astype(int)
y_test = H.y_tst.astype(int)
test_indices = np.random.choice(y_test.shape[0],test_size,replace=False)
X_test_small = X_test[test_indices]
y_test_small = y_test[test_indices]#
lab_size = 20
Ds = H

In [273]:
X_train = M.X_train
X_test = M.X_test
y_train = M.y_train.astype(int)
y_test = M.y_test.astype(int)
test_indices = np.random.choice(y_test.shape[0],test_size,replace=False)
X_test_small = X_test[test_indices]
y_test_small = y_test[test_indices]#
lab_size = 20
Ds = M

In [265]:

X_train = A.train_data.numpy()
m,s = X_train.mean(0),X_train.std(0)
X_test = At.test_data.numpy()#[:test_size]
y_train = A.train_labels.numpy().astype(int)
y_test = At.test_labels.numpy().astype(int)
test_indices = np.random.choice(y_test.shape[0],test_size,replace=False)
X_train = (X_train-m)/s
X_test = (X_test-m)/s
X_test_small = X_test[test_indices]
y_test_small = y_test[test_indices]#

lab_size = 200
Ds = A

In [305]:
X_train = Ya.X_train
m,s = X_train.mean(0),X_train.std(0)
X_test = Ya.X_test
y_train = Ya.y_train.astype(int)
y_test = Ya.y_test.astype(int)
test_indices = np.random.choice(y_test.shape[0],test_size,replace=False)
X_train = (X_train-m)/s
X_test = (X_test-m)/s
X_test_small = X_test[test_indices]
y_test_small = y_test[test_indices]#

lab_size = 800
Ds = Ya

In [306]:
with FixedNumpySeed(0):
    labIndices, devIndices = classBalancedSampleIndices(Ds, lab_size+5000, 5000)
    unlab_indices = np.random.choice(y_train.shape[0],unlab_size,replace=False)

Creating Train, Dev split         with 800 Train and 5000 Dev
Labeled indices hash 19929007


In [307]:
X_lab, X_dev = X_train[labIndices], X_train[devIndices]
y_lab, y_dev = y_train[labIndices], y_train[devIndices]

In [308]:

X_unlab = X_train[unlab_indices]
y_unlab = y_train[unlab_indices]

In [309]:
#X_unlab, X_lab, y_unlab, y_lab = train_test_split(X_train, y_train, test_size=lab_size,stratify=y_train)

In [310]:
#_, X_unlab, _, y_unlab = train_test_split(X_unlab, y_unlab, test_size=unlab_size,stratify=y_unlab)

In [311]:
X = np.concatenate((X_lab,X_unlab,X_test_small))
# l = np.arange(200)
# u = np.arange(200,1200)
l = np.arange(X.shape[0])<lab_size
u = (~l)&(np.arange(X.shape[0])<lab_size+X_unlab.shape[0])
t = np.arange(X.shape[0])>=lab_size+X_unlab.shape[0]


In [312]:
s=1/5
#dists = np.sum((X[:,None,:]-X[None,:,:])**2,axis=-1)/s**2
nx = np.linalg.norm(X,axis=1)
dists = 1-(X@X.T)/(nx[:,None]*nx[None,:])
W = np.exp(-dists*2)#-dists*100
W -= np.diag(np.diag(W))
#W += .00001*np.eye(X.shape[0])
D = np.diag(np.sum(W,axis=-1))
dm2 = np.sum(W,axis=-1)**-.5


In [313]:
L =dm2[:,None]*(D-.5*W)*dm2[None,:]
Y = np.zeros((X.shape[0],Ds.num_classes))
Y[:len(y_lab)] = oh(y_lab,Ds.num_classes)
Ys = np.linalg.solve(L,Y)
(Ys[t].argmax(-1)==y_test_small).mean()

0.24266666666666667

In [259]:
#plt.hist(np.log(nx))
#plt.xscale('log')

In [283]:
fu = -np.linalg.solve(L[u|t][:,u|t],(L[u|t][:,l]@oh(y_lab,Ds.num_classes)))
(fu[-X_test_small.shape[0]:].argmax(-1)==y_test_small).mean()

0.7823333333333333

In [272]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(X_lab, y_lab)
clf.score(X_test, y_test)

/home/marc_f/anaconda3/envs/default/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.7888157894736842

In [261]:
from sklearn.semi_supervised import LabelPropagation
label_prop_model = LabelPropagation()
partially_labels = np.concatenate((y_lab,y_unlab))
partially_labels[len(y_lab):] =-1
label_prop_model.fit(np.concatenate((X_lab,X_unlab)), partially_labels)

/home/marc_f/anaconda3/envs/default/lib/python3.6/site-packages/sklearn/semi_supervised/label_propagation.py:277: RuntimeWarning: divide by zero encountered in true_divide
  self.label_distributions_ /= normalizer
/home/marc_f/anaconda3/envs/default/lib/python3.6/site-packages/sklearn/semi_supervised/label_propagation.py:277: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
/home/marc_f/anaconda3/envs/default/lib/python3.6/site-packages/sklearn/semi_supervised/label_propagation.py:288: ConvergenceWarning: max_iter=1000 was reached without convergence.
  category=ConvergenceWarning


LabelPropagation(gamma=20, kernel='rbf', max_iter=1000, n_jobs=None,
                 n_neighbors=7, tol=0.001)

In [262]:
label_prop_model.score(X_test,y_test)

0.1

In [264]:
X_train.shape

(50000, 768)

In [ ]:
fu[-X_test.shape[0]:].argmax(-1)==y_test

In [ ]:
y_test.shape

In [ ]:
fu[-X_test.shape[0]:].argmax(-1).shape

In [ ]:
fu.shape

In [ ]:
np.where(u|t)

In [ ]:
np.where(t)

In [ ]:
t.shape

In [ ]:
fu[t].argmax(-1)==y_l

In [ ]:
+0*np.eye(X.shape[0]-lab_size)

In [ ]:
beta = np.linalg.solve(X[l].T@X[l]+100*np.eye(X.shape[1]),X[l].T@oh(y_lab,4))

In [ ]:
((X[l]@beta).argmax(-1)==y_lab).mean()

In [ ]:
((X[u]@beta).argmax(-1)==y_unlab).mean()

In [ ]:
((((x_test-m)/s)@beta).argmax(-1)==y_test).mean()